In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
df=pd.read_csv(r"C:\nydsa bootcamp slides\Projects\4\diabetic_data.csv")

In [10]:
# First focused on the column 24 to column 48. Those are the columns with the records of 25 types of medicines
original_df=df.copy()
original_df.T.iloc[24:49,]

,0,1,2,3,4,5,6,7,8,9,...,101756,101757,101758,101759,101760,101761,101762,101763,101764,101765
metformin,No,No,No,No,No,No,Steady,No,No,No,...,No,No,No,No,No,Steady,No,Steady,No,No
repaglinide,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
nateglinide,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
chlorpropamide,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
glimepiride,No,No,No,No,No,No,Steady,No,No,No,...,No,No,No,No,No,No,No,No,No,No
acetohexamide,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
glipizide,No,No,Steady,No,Steady,No,No,No,Steady,No,...,No,No,No,No,No,No,No,No,Steady,No
glyburide,No,No,No,No,No,No,No,Steady,No,No,...,No,No,No,No,No,No,No,No,No,No
tolbutamide,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
pioglitazone,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,Steady,No


In [32]:
#Count for each value 
med_col=original_df.iloc[:,24:47].apply(pd.value_counts).T
med_col

,Down,No,Steady,Up
metformin,575.0,81778.0,18346.0,1067.0
repaglinide,45.0,100227.0,1384.0,110.0
nateglinide,11.0,101063.0,668.0,24.0
chlorpropamide,1.0,101680.0,79.0,6.0
glimepiride,194.0,96575.0,4670.0,327.0
acetohexamide,NaN,101765.0,1.0,NaN
glipizide,560.0,89080.0,11356.0,770.0
glyburide,564.0,91116.0,9274.0,812.0
tolbutamide,NaN,101743.0,23.0,NaN
pioglitazone,118.0,94438.0,6976.0,234.0


In [33]:
#check the percentage of non-No values of each medicine.
# For columns has <0.1% of data, do we need to consider a drop?
temp=round(med_col[['Down','Steady','Up']].T.sum()/med_col.T.sum(),4)
sorted(zip(med_col.index,temp),key=lambda x:x[1],reverse=True)

[('insulin', 0.5344),
 ('metformin', 0.1964),
 ('glipizide', 0.1247),
 ('glyburide', 0.1047),
 ('pioglitazone', 0.072),
 ('rosiglitazone', 0.0625),
 ('glimepiride', 0.051),
 ('repaglinide', 0.0151),
 ('nateglinide', 0.0069),
 ('glyburide-metformin', 0.0069),
 ('acarbose', 0.003),
 ('chlorpropamide', 0.0008),
 ('miglitol', 0.0004),
 ('tolazamide', 0.0004),
 ('tolbutamide', 0.0002),
 ('glipizide-metformin', 0.0001),
 ('acetohexamide', 0.0),
 ('troglitazone', 0.0),
 ('examide', 0.0),
 ('citoglipton', 0.0),
 ('glimepiride-pioglitazone', 0.0),
 ('metformin-rosiglitazone', 0.0),
 ('metformin-pioglitazone', 0.0)]

# Does "No" mean not taking this kind of medicine or no impact after taking the medicine?
* acetohexamide -  has only one record
* chlorpropamide - has only 86 records
* tolbutamide - has only 23 records
* Similar situation to examide ,citoglipton, glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone

In [36]:
for name in original_df.columns:
    print (original_df[name].value_counts(),'\n')

96210942     1
89943846     1
384306986    1
94650156     1
83156784     1
2674482      1
281345844    1
193616274    1
355508024    1
165973818    1
125278944    1
420873188    1
157241154    1
161161032    1
174855390    1
134950734    1
154128210    1
96993108     1
122064144    1
297770840    1
382612616    1
165134172    1
108244830    1
210578766    1
443842340    1
151469730    1
289146210    1
154590960    1
145948404    1
176328594    1
            ..
249722520    1
111830682    1
126506652    1
80193186     1
186881700    1
147162726    1
263120844    1
249665124    1
151295556    1
113303472    1
73909806     1
422050106    1
13655088     1
168523320    1
296140568    1
98784828     1
157333056    1
280536642    1
130655706    1
190162530    1
107017800    1
103828530    1
176744010    1
172279374    1
297285200    1
74454612     1
208073976    1
166229592    1
38340702     1
77856768     1
Name: encounter_id, Length: 101766, dtype: int64 

88785891     40
43140906     28
23

None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64 

No        81778
Steady    18346
Up         1067
Down        575
Name: metformin, dtype: int64 

No        100227
Steady      1384
Up           110
Down          45
Name: repaglinide, dtype: int64 

No        101063
Steady       668
Up            24
Down          11
Name: nateglinide, dtype: int64 

No        101680
Steady        79
Up             6
Down           1
Name: chlorpropamide, dtype: int64 

No        96575
Steady     4670
Up          327
Down        194
Name: glimepiride, dtype: int64 

No        101765
Steady         1
Name: acetohexamide, dtype: int64 

No        89080
Steady    11356
Up          770
Down        560
Name: glipizide, dtype: int64 

No        91116
Steady     9274
Up          812
Down        564
Name: glyburide, dtype: int64 

No        101743
Steady        23
Name: tolbutamide, dtype: int64 

No        94438
Steady     6976
Up          234
Down        118
Name: pioglitaz

In [ ]:
# Race - 2273 ? value 
# Gender - 3 unknow/Invalid value
# Weight - 98569 ? value
# Payer_code - 40256 ? value
# medical_specialty - 49949 ? value
# diag_1 - 21 ? value
# diag_2 - 358 ? value
# diag_3 - 1423 ? value

In [28]:
original_df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [31]:
original_df[original_df['diag_1']=="?"]

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
518,4342662,1355697,Caucasian,Male,[60-70),?,6,25,1,4,...,No,No,No,No,No,No,No,No,No,NO
1006,7599132,9946782,?,Male,[80-90),?,3,3,5,3,...,No,No,No,No,No,No,No,No,No,>30
1267,8927178,520452,Caucasian,Male,[60-70),?,3,1,1,3,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
1488,10122996,3650130,AfricanAmerican,Male,[60-70),?,3,1,1,5,...,No,No,No,No,No,No,No,Ch,Yes,NO
3197,20095914,916947,AfricanAmerican,Male,[70-80),?,3,5,1,1,...,No,Steady,No,No,No,No,No,No,Yes,>30
7845,36370146,18328716,?,Male,[50-60),?,3,5,1,1,...,No,No,No,No,No,No,No,No,No,NO
14503,56481816,4232736,AfricanAmerican,Female,[20-30),?,1,18,7,6,...,No,Down,No,No,No,No,No,Ch,Yes,>30
19714,70017684,104694696,Caucasian,Male,[60-70),?,1,1,7,4,...,No,No,No,No,No,No,No,No,Yes,>30
32514,103561398,23460408,Caucasian,Male,[50-60),?,1,1,17,4,...,No,No,No,No,No,No,No,No,No,NO
37693,117010956,25300467,Hispanic,Male,[40-50),?,1,2,1,4,...,No,Steady,No,No,No,No,No,Ch,Yes,<30
